## Use Weighted MRT + Distance to Nearest Amenties
## Train Random Forest & Predict on Test Data

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
import category_encoders as ce
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from geopy.distance import geodesic
import geopy

from sklearn.model_selection import GridSearchCV
from sklearn import tree

In [2]:
# load pre-processed data
train_df = pd.read_csv('./data/full-train.csv', index_col=0)
test_df = pd.read_csv('./data/full-test.csv', index_col=0)
train_df

,type,bedrooms,bathrooms,district,planning_area,lat,lng,freehold,since_built_year,no_of_units,...,price,model_0,model_1,model_2,model_3,region_0,region_1,region_2,region_3,region_4
0,1,0.410302,1.493466,1.679353,1.623338,0.000000,0.504903,0,-0.452199,-0.751503,...,5390000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0,0.410302,0.624249,-0.111457,-0.297220,0.369401,0.539009,1,-0.051081,-1.054097,...,2310000.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1.317486,0.624249,-0.111457,-0.297220,0.351540,0.529769,1,-0.485626,-0.551361,...,5279500.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,0.410302,-0.244968,-0.939067,-0.806896,0.414697,0.749020,1,2.255354,-0.546596,...,1843600.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,0.410302,0.624249,0.042179,-0.806896,0.321693,0.768894,1,-0.619332,-1.025505,...,2262700.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,1,0.410302,0.624249,-0.735742,-0.483629,0.320402,0.596331,0,-0.318493,0.318300,...,2750000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26044,1,0.410302,0.624249,1.090614,0.959440,0.354335,0.460952,1,2.255354,-1.070775,...,6449300.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26045,1,-0.496883,-0.244968,-0.958805,-0.873456,0.469677,0.918986,0,-0.552479,-0.394108,...,1306800.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26046,1,1.317486,0.624249,-0.952210,-0.912355,0.443210,0.165124,0,-0.619332,0.647103,...,2420000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [3]:
# x y split
X_train = train_df.drop(columns = ['price'])
y_train = train_df['price']

X_test = test_df
X_test

,type,bedrooms,bathrooms,district,planning_area,lat,lng,freehold,since_built_year,no_of_units,...,additional_rooms,model_0,model_1,model_2,model_3,region_0,region_1,region_2,region_3,region_4
0,1,-0.496883,-0.244968,1.090614,-0.277340,0.341915,0.382426,1,1.252556,0.408840,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0.410302,-0.244968,0.042179,-0.483629,0.272228,0.672597,1,-0.619332,-0.801538,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0,-1.404067,-1.114184,0.042179,-0.806896,0.361653,0.760150,1,-0.519053,-0.737207,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,-0.496883,-1.114184,-1.001004,-0.912205,0.546606,0.963223,0,2.255354,-0.091513,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,1.317486,3.231899,1.159573,0.794445,0.175666,0.547546,0,-0.652759,0.103863,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,1,-1.404067,-1.114184,-0.603865,0.289945,0.231567,0.518446,0,-0.051081,0.060975,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7496,1,-0.496883,-0.244968,0.042179,-0.483629,0.263810,0.610629,0,0.015773,0.644720,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7497,0,0.410302,-0.244968,-1.017347,-0.952783,0.626063,0.219142,0,2.255354,-0.934965,...,0.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7498,1,-0.496883,-1.114184,-1.076004,-0.891256,0.733540,0.637659,0,-0.786465,0.639955,...,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
weighted_distance = pd.read_csv('./data/auxiliary-mrt-distance-weighted.csv', index_col=0)
weighted_distance_test = pd.read_csv('./data/auxiliary-mrt-distance-weighted-test.csv', index_col=0)

In [5]:
# standardize auxiliary data
weighted_betweenness_distance = weighted_distance.drop(columns = ['closeness_closest_mrt', 'closest_mrt'])

num_scaler = StandardScaler()
weighted_betweenness_distance[weighted_betweenness_distance.columns] = num_scaler.fit_transform(weighted_betweenness_distance) 

In [6]:
# transform standardization to test data
weighted_betweenness_distance_test = weighted_distance_test.drop(columns = ['closeness_closest_mrt', 'closest_mrt'])
weighted_betweenness_distance_test[weighted_betweenness_distance_test.columns] = num_scaler.fit_transform(weighted_betweenness_distance_test)

In [7]:
x_train_distance_betweenness = X_train.join(weighted_betweenness_distance)
x_train_distance_betweenness

,type,bedrooms,bathrooms,district,planning_area,lat,lng,freehold,since_built_year,no_of_units,...,region_1,region_2,region_3,region_4,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall
0,1,0.410302,1.493466,1.679353,1.623338,0.000000,0.504903,0,-0.452199,-0.751503,...,0.0,0.0,0.0,0.0,-0.588448,3.638933,4.302719,5.949749,5.132038,4.416372
1,0,0.410302,0.624249,-0.111457,-0.297220,0.369401,0.539009,1,-0.051081,-1.054097,...,0.0,0.0,0.0,0.0,-0.365250,-1.352215,-0.612578,-0.984931,0.433492,-0.696530
2,1,1.317486,0.624249,-0.111457,-0.297220,0.351540,0.529769,1,-0.485626,-0.551361,...,0.0,0.0,0.0,0.0,-0.440583,-1.267173,-0.433263,-0.875986,0.149582,-0.943701
3,1,0.410302,-0.244968,-0.939067,-0.806896,0.414697,0.749020,1,2.255354,-0.546596,...,1.0,0.0,0.0,0.0,-0.252795,0.035690,0.080960,-0.530506,-0.716548,1.696195
4,0,0.410302,0.624249,0.042179,-0.806896,0.321693,0.768894,1,-0.619332,-1.025505,...,1.0,0.0,0.0,0.0,-0.502415,0.398764,-0.474028,-0.728795,-0.523390,0.163528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,1,0.410302,0.624249,-0.735742,-0.483629,0.320402,0.596331,0,-0.318493,0.318300,...,0.0,0.0,0.0,0.0,0.335776,0.190992,-0.848122,0.138355,1.154342,-1.079324
26044,1,0.410302,0.624249,1.090614,0.959440,0.354335,0.460952,1,2.255354,-1.070775,...,0.0,0.0,0.0,0.0,0.119241,0.250770,0.755159,-0.298392,-0.543896,0.625791
26045,1,-0.496883,-0.244968,-0.958805,-0.873456,0.469677,0.918986,0,-0.552479,-0.394108,...,1.0,0.0,0.0,0.0,-0.358672,-0.648330,0.171497,-1.042741,-1.310364,-0.731834
26046,1,1.317486,0.624249,-0.952210,-0.912355,0.443210,0.165124,0,-0.619332,0.647103,...,0.0,0.0,0.0,1.0,1.012329,-0.882430,0.080062,-0.188887,-0.517657,-0.815260


In [9]:
x_test_distance_betweenness = X_test.join(weighted_betweenness_distance_test)

In [10]:
x_test_distance_betweenness

,type,bedrooms,bathrooms,district,planning_area,lat,lng,freehold,since_built_year,no_of_units,...,region_1,region_2,region_3,region_4,betweenness_closest_mrt,closest_comercial,closest_hawker,closest_primary,closest_secondary,closest_mall
0,1,-0.496883,-0.244968,1.090614,-0.277340,0.341915,0.382426,1,1.252556,0.408840,...,0.0,0.0,0.0,0.0,0.138464,-0.142911,-0.766006,0.394425,0.118714,0.742840
1,1,0.410302,-0.244968,0.042179,-0.483629,0.272228,0.672597,1,-0.619332,-0.801538,...,0.0,0.0,0.0,0.0,-0.559139,0.582041,-0.716975,1.303927,-1.104269,0.452027
2,0,-1.404067,-1.114184,0.042179,-0.806896,0.361653,0.760150,1,-0.519053,-0.737207,...,1.0,0.0,0.0,0.0,-0.360056,-0.088743,-0.514987,0.232289,0.568633,1.357887
3,1,-0.496883,-1.114184,-1.001004,-0.912205,0.546606,0.963223,0,2.255354,-0.091513,...,1.0,0.0,0.0,0.0,-0.535067,0.044608,1.944232,-0.111140,-0.674561,2.892877
4,1,1.317486,3.231899,1.159573,0.794445,0.175666,0.547546,0,-0.652759,0.103863,...,0.0,0.0,0.0,0.0,-0.437737,-0.614000,-0.882432,0.580318,1.090210,-0.593629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,1,-1.404067,-1.114184,-0.603865,0.289945,0.231567,0.518446,0,-0.051081,0.060975,...,0.0,0.0,0.0,0.0,-0.398095,-0.556234,-1.059415,-0.025107,-0.580058,-0.528196
7496,1,-0.496883,-0.244968,0.042179,-0.483629,0.263810,0.610629,0,0.015773,0.644720,...,0.0,0.0,0.0,0.0,-0.501897,-0.035278,-0.244673,1.092195,1.121171,0.364316
7497,0,0.410302,-0.244968,-1.017347,-0.952783,0.626063,0.219142,0,2.255354,-0.934965,...,0.0,0.0,0.0,1.0,-0.337995,2.651082,1.044666,0.070949,-0.379996,-0.153165
7498,1,-0.496883,-1.114184,-1.076004,-0.891256,0.733540,0.637659,0,-0.786465,0.639955,...,0.0,0.0,1.0,0.0,-0.613433,-0.315685,2.573472,-0.254157,-0.047421,0.087240


In [11]:
best_param = {'bootstrap': True, 'max_depth': 50, 'max_features': None, 'min_samples_split': 5, 'n_estimators': 100}
rfr = RandomForestRegressor()
rfr.set_params(**best_param)

RandomForestRegressor(max_depth=50, max_features=None, min_samples_split=5)

In [28]:
rfr.fit(x_train_distance_betweenness, y_train)

RandomForestRegressor(max_depth=50, max_features=None, min_samples_split=5)

In [29]:
result = rfr.predict(x_test_distance_betweenness)

In [30]:
result

array([2039623.39920635, 2600708.9399429 ,  763280.23678266, ...,
       1479897.43611111, 1010856.43840049, 7864203.87539683])

In [31]:
ID = range(0, len(result))

In [32]:
df = pd.DataFrame(ID, columns = ['Id'])

In [37]:
# df['Predicted'] = np.round(result, 1)
df['Predicted'] = result

In [38]:
df

,Id,Predicted
0,0,2.039623e+06
1,1,2.600709e+06
2,2,7.632802e+05
3,3,9.911282e+05
4,4,2.007031e+07
...,...,...
7495,7495,1.302196e+06
7496,7496,2.084445e+06
7497,7497,1.479897e+06
7498,7498,1.010856e+06


In [39]:
# save predicted value
path = './data/random-forest-betweenness-distance-predict.csv'
df.to_csv(path, index = False)